# Script para ler o Diário Oficial do Município de São Paulo

## Script BASE

### Importando as bibliotecas

In [199]:
from datetime import date, datetime, timedelta
from PyPDF2 import PdfFileReader, PdfFileMerger
from collections import OrderedDict
import os
import fitz as fz
import tabula
import scripts

### Obter data atual

In [2]:
def get_date_today():
    '''Obtém a data atual e retorna uma string: (YYYY-mm-dd)'''
    dt_now = date.today()
    if dt_now.weekday() == 0:
            dt_now = dt_now + timedelta(-2)
            dt_now = dt_now.strftime("%Y-%m-%d")
    else:
            dt_now = dt_now.strftime("%Y-%m-%d")
    
    return dt_now

dt_now = get_date_today()

### Criando o diretório SAVE_DIR

In [3]:
BASE_DIR = os.getcwd()
os.chdir(BASE_DIR)
try:
    os.makedirs(f'{BASE_DIR}\\outputs_files\\{dt_now}')
    print(f'### MAKEDIRS CONCLUÍDO - Verifique a pasta: {BASE_DIR}\\outputs_files\\{dt_now}')
except:
    print(f'### MAKEDIRS PASTA EXISTENTE - Verifique a pasta: {BASE_DIR}\\outputs_files\\{dt_now}')
    pass

### MAKEDIRS CONCLUÍDO - Verifique a pasta: c:\Projetos\scrapy_dom\outputs_files\2022-05-31


### Obter diretórios

In [8]:
def get_directories():
        '''Obtém  os diretórios de trabalho.
                return BASE_DIR,DATA_DIR,SAVE_DIR,DATA_FILE,DATA_FILES
        '''
        dt_now = get_date_today()
        BASE_DIR = os.getcwd()
        os.chdir(BASE_DIR)
        try:
                os.makedirs(f'{BASE_DIR}\\outputs_files\\{dt_now}')
        except:
                pass

        DATA_DIR = f'{BASE_DIR}\\querido-diario\\data_collection\\data\\3550308\\{str(dt_now)}'
        SAVE_DIR = f'{BASE_DIR}\\outputs_files\\{dt_now}'
        # DATA_FILE = SAVE_DIR + '\\' + [f for f in os.listdir(SAVE_DIR) if f.endswith("tsv")][0]
        DATA_FILES = os.listdir(DATA_DIR)
        DATA_FILES.sort()
        
        print(f'###   INFO:  DIRS   ###')
        print(f'  BASE_DIR:  {BASE_DIR}')
        print(f'  DATA_DIR:  {DATA_DIR}')
        print(f'  SAVE_DIR:  {SAVE_DIR}')
        #print(f' DATA_FILE:  {DATA_FILE}')
        print(f'DATA_FILES:  {DATA_FILES}')
        
        return BASE_DIR,DATA_DIR,SAVE_DIR,DATA_FILES

BASE_DIR,DATA_DIR,SAVE_DIR,DATA_FILES = get_directories()

###   INFO:  DIRS   ###
  BASE_DIR:  c:\Projetos\scrapy_dom
  DATA_DIR:  c:\Projetos\scrapy_dom\querido-diario\data_collection\data\3550308\2022-05-31
  SAVE_DIR:  c:\Projetos\scrapy_dom\outputs_files\2022-05-31
DATA_FILES:  ['00dfc087760fc911002cb83032fb4394dfc39d55.pdf', '02948d080f8ff3e9513fd86117106d41009d291d.pdf', '05ea73c739e19a1b47b2c258652c903145bd5e74.pdf', '06c54d6a098ef58e0c339ae8ecc0fc1cd5f52050.pdf', '0946ed41292871f7b8549df2af056bc1a4260cd2.pdf', '0a1c34d3562d9d9e80ea3686a8fe174447adc688.pdf', '0b1d06b41fe436e0bdae5e5c919f877a6dc5f6bc.pdf', '0bc126f9dbd255f3443d581bc2a27891fc5ef054.pdf', '0be25fdab75fabef9e1340b78061a317ab562067.pdf', '0d6cfdf9160e51df6d42f4cb48aefa5a95bdf4d7.pdf', '0e4cc1477d47c1c7796ae44a64493ce503dbbc8f.pdf', '0f09d50f2339556f2a59ed4e6eabebfc3c0801cf.pdf', '0f1836bf934633bc3207ce52abd412d0a1a99dc7.pdf', '13eaee65f204a92436d50003bb6789ddeb5e875a.pdf', '14fb1b3addf6950251a3f137cc27b5c2a45b817d.pdf', '1770c3eb265ffa1d3d1a1d2593d9474c99e3efe7.pdf', '19025

### Obtendo Informações do DOM

In [14]:
def get_info_dom():
        '''Obtém as informações do DOM.
                return YEAR_DOM,EDITION_DOM,PAGS_DOM,HEAD_DOM        
        '''
        dt_now = get_date_today()
        BASE_DIR,DATA_DIR,SAVE_DIR,DATA_FILES = get_directories()
        YEAR_DOM = date.today().year - 1955
        HEAD_DOM ='São Paulo, ' + str(YEAR_DOM) + ' ('

        with fz.open(DATA_DIR +'\\'+ DATA_FILES[3]) as file:
                search_edition = ''
                for page in file:
                        search_edition += page.get_text()

        search_edition = search_edition[search_edition.find(HEAD_DOM):]
        EDITION_DOM = search_edition[search_edition.find('(')+1:search_edition.find(')')]
        HEAD_DOM ='São Paulo, ' + str(YEAR_DOM) + ' (' + EDITION_DOM + ')'
        PAGS_DOM = str(len(DATA_FILES))
        DOM = f'DOM_SP-{str(YEAR_DOM)}_{str(EDITION_DOM)}_{PAGS_DOM}_{str(dt_now)}'

        print('### INFO: DOM-SP ###')
        print(f'DATA: {dt_now}')
        print(f'PAGS: {PAGS_DOM}')
        print(f'HEAD: {HEAD_DOM}')
        print(f' DOM: {DOM}')
        
        return YEAR_DOM,EDITION_DOM,PAGS_DOM,HEAD_DOM,DOM

YEAR_DOM,EDITION_DOM,PAGS_DOM,HEAD_DOM,DOM = get_info_dom()

###   INFO:  DIRS   ###
  BASE_DIR:  c:\Projetos\scrapy_dom
  DATA_DIR:  c:\Projetos\scrapy_dom\querido-diario\data_collection\data\3550308\2022-05-31
  SAVE_DIR:  c:\Projetos\scrapy_dom\outputs_files\2022-05-31
DATA_FILES:  ['00dfc087760fc911002cb83032fb4394dfc39d55.pdf', '02948d080f8ff3e9513fd86117106d41009d291d.pdf', '05ea73c739e19a1b47b2c258652c903145bd5e74.pdf', '06c54d6a098ef58e0c339ae8ecc0fc1cd5f52050.pdf', '0946ed41292871f7b8549df2af056bc1a4260cd2.pdf', '0a1c34d3562d9d9e80ea3686a8fe174447adc688.pdf', '0b1d06b41fe436e0bdae5e5c919f877a6dc5f6bc.pdf', '0bc126f9dbd255f3443d581bc2a27891fc5ef054.pdf', '0be25fdab75fabef9e1340b78061a317ab562067.pdf', '0d6cfdf9160e51df6d42f4cb48aefa5a95bdf4d7.pdf', '0e4cc1477d47c1c7796ae44a64493ce503dbbc8f.pdf', '0f09d50f2339556f2a59ed4e6eabebfc3c0801cf.pdf', '0f1836bf934633bc3207ce52abd412d0a1a99dc7.pdf', '13eaee65f204a92436d50003bb6789ddeb5e875a.pdf', '14fb1b3addf6950251a3f137cc27b5c2a45b817d.pdf', '1770c3eb265ffa1d3d1a1d2593d9474c99e3efe7.pdf', '19025

### Renomeando arquivos

In [16]:
contexts = ''
for file_name in DATA_FILES:
        print('Obtendo conteúdo de: ' + file_name)
        with fz.open(DATA_DIR +'\\'+ file_name) as file:
                context = ""
                for page in file:
                        context += '<pag>\n'+ page.get_text() + '</pag>\n'
                
                odd_page = "São Paulo, " + str(YEAR_DOM) + " (" + str(EDITION_DOM) + ") – "
                even_page = " – São Paulo, " + str(YEAR_DOM) + " (" + str(EDITION_DOM) + ")"

                if  context.find(odd_page) != -1:
                        num_page = context[context.find(odd_page)+21:context.find('\n',context.find(odd_page))]
                elif context.find(even_page) != -1:
                        num_page = context[context.find('\n',context.find(even_page)-5)+1:context.find(even_page)]
                elif context.find("D.O.C.; " + HEAD_DOM) != -1:
                        num_page = str(len(DATA_FILES))
                elif context.find("Ano "+ str(YEAR_DOM)) != -1:
                        num_page = '1'
                else:
                        num_page = ''

        contexts += context
        os.rename(f'{DATA_DIR}//{file_name}',f'{DATA_DIR}\\pg_{(num_page.strip()).zfill(3)}-{file_name}')

DATA_FILES = os.listdir(DATA_DIR)
DATA_FILES.sort()
print(f'### RENAME CONCLUÍDO - Verifique a pasta: {DATA_DIR} ###')

Obtendo conteúdo de: 00dfc087760fc911002cb83032fb4394dfc39d55.pdf
Obtendo conteúdo de: 02948d080f8ff3e9513fd86117106d41009d291d.pdf
Obtendo conteúdo de: 05ea73c739e19a1b47b2c258652c903145bd5e74.pdf
Obtendo conteúdo de: 06c54d6a098ef58e0c339ae8ecc0fc1cd5f52050.pdf
Obtendo conteúdo de: 0946ed41292871f7b8549df2af056bc1a4260cd2.pdf
Obtendo conteúdo de: 0a1c34d3562d9d9e80ea3686a8fe174447adc688.pdf
Obtendo conteúdo de: 0b1d06b41fe436e0bdae5e5c919f877a6dc5f6bc.pdf
Obtendo conteúdo de: 0bc126f9dbd255f3443d581bc2a27891fc5ef054.pdf
Obtendo conteúdo de: 0be25fdab75fabef9e1340b78061a317ab562067.pdf
Obtendo conteúdo de: 0d6cfdf9160e51df6d42f4cb48aefa5a95bdf4d7.pdf
Obtendo conteúdo de: 0e4cc1477d47c1c7796ae44a64493ce503dbbc8f.pdf
Obtendo conteúdo de: 0f09d50f2339556f2a59ed4e6eabebfc3c0801cf.pdf
Obtendo conteúdo de: 0f1836bf934633bc3207ce52abd412d0a1a99dc7.pdf
Obtendo conteúdo de: 13eaee65f204a92436d50003bb6789ddeb5e875a.pdf
Obtendo conteúdo de: 14fb1b3addf6950251a3f137cc27b5c2a45b817d.pdf
Obtendo co

In [17]:
def get_directories():
        '''Obtém  os diretórios de trabalho.
                return BASE_DIR,DATA_DIR,SAVE_DIR,DATA_FILE,DATA_FILES
        '''
        dt_now = get_date_today()
        BASE_DIR = os.getcwd()
        os.chdir(BASE_DIR)
        try:
                os.makedirs(f'{BASE_DIR}\\outputs_files\\{dt_now}')
        except:
                pass

        DATA_DIR = f'{BASE_DIR}\\querido-diario\\data_collection\\data\\3550308\\{str(dt_now)}'
        SAVE_DIR = f'{BASE_DIR}\\outputs_files\\{dt_now}'
        DATA_FILE = SAVE_DIR + '\\' + [f for f in os.listdir(SAVE_DIR) if f.endswith("tsv")][0]
        DATA_FILES = os.listdir(DATA_DIR)
        DATA_FILES.sort()
        
        print(f'###   INFO:  DIRS   ###')
        print(f'  BASE_DIR:  {BASE_DIR}')
        print(f'  DATA_DIR:  {DATA_DIR}')
        print(f'  SAVE_DIR:  {SAVE_DIR}')
        print(f' DATA_FILE:  {DATA_FILE}')
        print(f'DATA_FILES:  {DATA_FILES}')
        
        return BASE_DIR,DATA_DIR,SAVE_DIR,DATA_FILE,DATA_FILES

BASE_DIR,DATA_DIR,SAVE_DIR,DATA_FILE,DATA_FILES = get_directories()

IndexError: list index out of range

### Juntando os arquivos

In [18]:
pdf_files = [f for f in os.listdir(DATA_DIR) if f.endswith("pdf")]
merger = PdfFileMerger()

for filename in pdf_files:
    print(f'Merge de: {filename}')
    merger.append(PdfFileReader(os.path.join(DATA_DIR, filename), "rb"))

merger.write(SAVE_DIR + "\\DOM_SP-"+ str(YEAR_DOM) + "_" + str(EDITION_DOM) + "_" + str(len(DATA_FILES)) + "_" + str(dt_now)+".pdf")
print(f'### MERGE CONCLUÍDO - Verifique a pasta: {SAVE_DIR} ###')

Merge de: pg_001-c664bf661633cf387f40656ff590a297c450ef53.pdf
Merge de: pg_002-906b68dbe9153587cf2c5db74e71a417871f7f1d.pdf
Merge de: pg_003-8a7e5ef2636dff9e0dca5672894f41ab3065e5b3.pdf
Merge de: pg_004-d4f1cae23565faa860bf86ee146d32a7b15a8363.pdf
Merge de: pg_005-7cc4619cd9a0dcbc01db25af36b53b575ad3f4be.pdf
Merge de: pg_006-0d6cfdf9160e51df6d42f4cb48aefa5a95bdf4d7.pdf
Merge de: pg_007-a768f94c708363fe36b765997e6c3353e91c8f0a.pdf
Merge de: pg_008-bc78ba3863b896205f892bc82eb7a5a5e8247728.pdf
Merge de: pg_009-a6e1b00b58e7a074891f620d4f52f83540b39ca2.pdf
Merge de: pg_010-d50a630e92646374f9868b191edceddc5dde03ed.pdf
Merge de: pg_011-baf1c0d418422038013966f3abbb6f2bd1349bba.pdf
Merge de: pg_012-9458feab241871a23d3d578e5dcc1322ba189068.pdf
Merge de: pg_013-7cc212272136a479ef0a620a2874a9c141b1a900.pdf
Merge de: pg_014-9ef7057a5b7650b623075f490071b0e903324b51.pdf
Merge de: pg_015-2d182b79f59e59dbd75060a901c08afd875a66b8.pdf
Merge de: pg_016-c967d06484cde9dd5df954b33fc3cc0a27bbb4ee.pdf
Merge de

### Convertendo para TSV FINAL

In [19]:
print('Convertendo arquivo para TSV')
area_text = [[42.849,28.172,1209.134,215.329],[42.849,216.314,1210.119,393.621],[42.849,395.591,1210.119,574.868],[42.849,576.838,1207.164,760.055]]
tabula.convert_into_by_batch(SAVE_DIR,output_format="tsv",area=area_text,pages="all")
print(f'### CONVERT CONCLUÍDO - Verifique a pasta: {SAVE_DIR} ###')

Convertendo arquivo para TSV


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xca in position 84: invalid continuation byte

### Salva arquivo final

In [20]:
def save_file(SAVE_DIR,CONTEXT,NAME_FILE,FORMAT='txt'):
        '''
                Cria um arquivo com o conteúdo e salva na pasta.
                        SAVE_DIR  : Diretório para salvar o conteúdo
                        CONTEXT   : Conteúdo a ser salvo
                        NAME_FILE : Nome do arquivo para ser salvo
                        FORMAT    : Formato do arquivo final
        '''

        with open(f"{SAVE_DIR}\\{NAME_FILE}.{FORMAT}","w",encoding='utf8') as output_file:
                output_file.write(CONTEXT)
        
        return f"{SAVE_DIR}\\{NAME_FILE}.{FORMAT}"

### Definindo Diretórios

### Obtem contéudo do arquivo TSV

In [213]:
with open(DATA_FILE,'r') as file:
    conteudo = file.read()

## Scripts de Listagens

### Seção SERVIDORES

In [214]:
conteudo_alterado = conteudo

#### Limpeza dos dados

In [215]:
conteudo_alterado = conteudo_alterado.replace('\t',' ')
conteudo_alterado = conteudo_alterado.replace('""','')
conteudo_alterado = conteudo_alterado.replace(' \n','\n')
conteudo_alterado = conteudo_alterado.replace('\n ','\n')
conteudo_alterado = conteudo_alterado.replace('    ',' ')
conteudo_alterado = conteudo_alterado.replace('   ',' ')
conteudo_alterado = conteudo_alterado.replace('  ',' ')
conteudo_alterado = conteudo_alterado.replace('-\n','')
conteudo_alterado = conteudo_alterado.replace(',\n',', ')
conteudo_alterado = conteudo_alterado.replace(', \n',', ')
conteudo_alterado = conteudo_alterado.replace('/\n','/')
conteudo_alterado = conteudo_alterado.upper()
conteudo_alterado = conteudo_alterado.replace('FUNDAÇÃO PAULISTANA DE EDUCAÇÃO\nTECNOLOGIA E CULTURA','FUNDAÇÃO PAULISTANA DE EDUCAÇÃO TECNOLOGIA E CULTURA')
conteudo_alterado = conteudo_alterado.replace('HOSPITAL DO SERVIDOR\nPÚBLICO MUNICIPAL','HOSPITAL DO SERVIDOR PÚBLICO MUNICIPAL')
conteudo_alterado = conteudo_alterado.replace('SECRETARIA MUNICIPAL DE ASSIST E DESENV\nSOCIAL','SECRETARIA MUNICIPAL DE ASSIST E DESENV SOCIAL')
conteudo_alterado = conteudo_alterado.replace('SECRETARIA MUNICIPAL DE INFRAESTRUTURA\nURBANA E OB','SECRETARIA MUNICIPAL DE INFRAESTRUTURA URBANA E OB')
conteudo_alterado = conteudo_alterado.replace('SECRETARIA MUNICIPAL DE SEGURANCA\nURBANA','SECRETARIA MUNICIPAL DE SEGURANCA URBANA')
conteudo_alterado = conteudo_alterado.replace('SECRETARIA MUNICIPAL DO VERDE E DO MEIO\nAMBIENTE','SECRETARIA MUNICIPAL DO VERDE E DO MEIO AMBIENTE')
conteudo_alterado = conteudo_alterado.replace('COORDENADORIA DE ADMINISTRAÇÃO\nE FINANÇAS','COORDENADORIA DE ADMINISTRAÇÃO E FINANÇAS')
conteudo_alterado = conteudo_alterado.replace('LICENÇAS MÉDICAS CONCEDIDAS/NEGADAS\nEM GRAU DE RECURSO','LICENÇAS MÉDICAS CONCEDIDAS/NEGADAS EM GRAU DE RECURSO')
conteudo_alterado = conteudo_alterado.replace('RELAÇÃO DE LICENÇAS MÉDICAS NEGADAS\nNOS TERMOS DA LEI 8989/79','RELAÇÃO DE LICENÇAS MÉDICAS NEGADAS NOS TERMOS DA LEI 8989/79')
conteudo_alterado = conteudo_alterado.replace('RELAÇÃO DE FALTAS DOS CONVOCADOS\nPARA AVALIAÇÃO DE ESPECIALISTA','RELAÇÃO DE FALTAS DOS CONVOCADOS PARA AVALIAÇÃO DE ESPECIALISTA')
conteudo_alterado = conteudo_alterado.replace('RELAÇÃO DE FALTAS DOS CONVOCADOS\nPARA JUNTA MÉDICA PARA FINS DE PENSÃO\nPOR MORTE','RELAÇÃO DE FALTAS DOS CONVOCADOS PARA JUNTA MÉDICA PARA FINS DE PENSÃO POR MORTE')
conteudo_alterado = conteudo_alterado.replace('RELAÇÃO DE ALTA DE AT/DT RETIFICADA EM\nFUNÇÃO DE:','RELAÇÃO DE ALTA DE AT/DT RETIFICADA EM FUNÇÃO DE:')
conteudo_alterado = conteudo_alterado.replace('RELAÇÃO DE SERVIDORES SUBMETIDOS À\nAVALIAÇÃO DE ESPECIALISTA\nCOM CAPACIDADE LABORATIVA PARA A SUA\nFUNÇÃO ORIGINAL','RELAÇÃO DE SERVIDORES SUBMETIDOS À AVALIAÇÃO DE ESPECIALISTA COM CAPACIDADE LABORATIVA PARA A SUA FUNÇÃO ORIGINAL')
conteudo_alterado = conteudo_alterado.replace('RELAÇÃO DE CONCESSÃO DE NEXO DE ACIDENTE DO TRABALHO\nDEFERIDO','RELAÇÃO DE CONCESSÃO DE NEXO DE ACIDENTE DO TRABALHO DEFERIDO')
conteudo_alterado = conteudo_alterado.replace('LICENÇAS MÉDICAS DE CURTA DURAÇÃO\nDOC ','LICENÇAS MÉDICAS DE CURTA DURAÇÃO DOC ')
conteudo_alterado = conteudo_alterado.replace('ACI-\nDENTE','ACIDENTE')
conteudo_alterado = conteudo_alterado.replace('CANCELA-\nDAS','CANCELADAS')
conteudo_alterado = conteudo_alterado.replace('RETIFICA-\nDAS','RETIFICADAS')
conteudo_alterado = conteudo_alterado.replace('RECO-\nMENDAÇÃO','RECOMENDAÇÃO')
conteudo_alterado = conteudo_alterado.replace('RECOMENDAÇÃO\nD','RECOMENDAÇÃO D')
conteudo_alterado = conteudo_alterado.replace('LICENÇAS MÉDICAS\nFACE','LICENÇAS MÉDICAS FACE')
conteudo_alterado = conteudo_alterado.replace('LICENÇAS MÉDICAS PARA SERVIDORES SOB REGIME GERAL DA\nPREVIDÊNCIA SOCIAL\nCONCEDIDA DE ACORDO COM O DETERMINADO NA PORTARIA NO\n','LICENÇAS MÉDICAS PARA SERVIDORES SOB REGIME GERAL DA PREVIDÊNCIA SOCIAL CONCEDIDA DE ACORDO COM O DETERMINADO NA PORTARIA NO ')
conteudo_alterado = conteudo_alterado.replace('LICENÇA\nMÉDICA','LICENÇA MÉDICA')
conteudo_alterado = conteudo_alterado.replace('DECRETO NO\n','DECRETO NO')
conteudo_alterado = conteudo_alterado.replace('PUBLICADO\nNO','PUBLICADO NO')
conteudo_alterado = conteudo_alterado.replace('\nCONCEDIDA',' CONCEDIDA')
conteudo_alterado = conteudo_alterado.replace('\nCONCEDIDAS ',' CONCEDIDAS ')
conteudo_alterado = conteudo_alterado.replace(' RETIFICA- DAS ',' RETIFICADAS ')
conteudo_alterado = conteudo_alterado.replace('PORTARIA\n','PORTARIA ')
conteudo_alterado = conteudo_alterado.replace(' TRABALHO\nINDEFERIDO',' TRABALHO INDEFERIDO')
conteudo_alterado = conteudo_alterado.replace(' DOC\nSEI ',' DOC SEI ')
conteudo_alterado = conteudo_alterado.replace(' DA LEI\n',' DA LEI ')
conteudo_alterado = conteudo_alterado.replace('\nAOS ',' AOS ')
conteudo_alterado = conteudo_alterado.replace(' SÃO\n',' SÃO ')
conteudo_alterado = conteudo_alterado.replace(' À\n',' À ')
conteudo_alterado = conteudo_alterado.replace(' ÀS\n',' ÀS ')
conteudo_alterado = conteudo_alterado.replace(' AS\n',' AS ')
conteudo_alterado = conteudo_alterado.replace(' E\n',' E ')
conteudo_alterado = conteudo_alterado.replace(' O\n',' O ')
conteudo_alterado = conteudo_alterado.replace(' OS\n',' OS ')
conteudo_alterado = conteudo_alterado.replace('\nOS ',' OS ')
conteudo_alterado = conteudo_alterado.replace(' DA\n',' DA ')
conteudo_alterado = conteudo_alterado.replace('\nDA ',' DA ')
conteudo_alterado = conteudo_alterado.replace(' DAS\n',' DAS ')
conteudo_alterado = conteudo_alterado.replace('\nDAS ',' DAS ')
conteudo_alterado = conteudo_alterado.replace(' DA(S)\n',' DA(S) ')
conteudo_alterado = conteudo_alterado.replace('\nDA(S) ',' DA(S) ')
conteudo_alterado = conteudo_alterado.replace(' DE\n',' DE ')
conteudo_alterado = conteudo_alterado.replace('\nDE ',' DE ')
conteudo_alterado = conteudo_alterado.replace(' DO\n',' DO ')
conteudo_alterado = conteudo_alterado.replace('\nDO ',' DO ')
conteudo_alterado = conteudo_alterado.replace(' DOS\n',' DOS ')
conteudo_alterado = conteudo_alterado.replace('\nDOS ',' DOS ')
conteudo_alterado = conteudo_alterado.replace(' DO(S)\n',' DO(S) ')
conteudo_alterado = conteudo_alterado.replace('\nDO(S) ',' DO(S) ')
conteudo_alterado = conteudo_alterado.replace(' COM\n',' COM ')
conteudo_alterado = conteudo_alterado.replace('\nCOM ',' COM ')
conteudo_alterado = conteudo_alterado.replace(' EM\n',' EM ')
conteudo_alterado = conteudo_alterado.replace('\nEM ',' EM ')
conteudo_alterado = conteudo_alterado.replace(' PARA\n',' PARA ')
conteudo_alterado = conteudo_alterado.replace('\nPARA ',' PARA ')
conteudo_alterado = conteudo_alterado.replace(' POR\n',' POR ')
conteudo_alterado = conteudo_alterado.replace(' NA\n',' NA ')
conteudo_alterado = conteudo_alterado.replace(' NAS\n',' NAS ')
conteudo_alterado = conteudo_alterado.replace('\nNAS ',' NAS ')
conteudo_alterado = conteudo_alterado.replace(' NO\n',' NO ')
conteudo_alterado = conteudo_alterado.replace(' NOS\n',' NOS ')
conteudo_alterado = conteudo_alterado.replace('\nNOS ',' NOS ')
conteudo_alterado = conteudo_alterado.replace('A PARTIR DE',' A_PARTIR_DE')
conteudo_alterado = conteudo_alterado.replace('RF NOME DIAS A PARTIR DE ARTIGO','RF NOME DIAS A_PARTIR_DE ARTIGO')
conteudo_alterado = conteudo_alterado.replace('RF NOME A_PARTIR_DE ','RF NOME A_PARTIR_DE\n')
conteudo_alterado = conteudo_alterado.replace('REG. FUNC. NOME CONTROLE NO ','RF NOME CONTROLE_NO\n')
conteudo_alterado = conteudo_alterado.replace('R.F./V NOME DIAS/EXERCÍCIO A_PARTIR_DE','RF/V NOME DIAS/EXERCÍCIO A_PARTIR_DE')
conteudo_alterado = conteudo_alterado.replace('REG. FUNC/VINC NOME NÍVEL CAT. SÍMBOLO A_PARTIR_DE','RF VINC NOME NÍVEL_CAT SÍMBOLO A_PARTIR_DE')
conteudo_alterado = conteudo_alterado.replace('RF V NOME EH A PARTIR MOTIVO','RF VINC NOME EH A_PARTIR MOTIVO')

In [216]:
conteudo_alterado = conteudo_alterado.replace('\t',' ')
conteudo_alterado = conteudo_alterado.replace('""','')
conteudo_alterado = conteudo_alterado.replace(' \n','\n')
conteudo_alterado = conteudo_alterado.replace('\n ','\n')
conteudo_alterado = conteudo_alterado.replace('    ',' ')
conteudo_alterado = conteudo_alterado.replace('   ',' ')
conteudo_alterado = conteudo_alterado.replace('  ',' ')
conteudo_alterado = conteudo_alterado.replace('-\n','')
conteudo_alterado = conteudo_alterado.replace(',\n',', ')
conteudo_alterado = conteudo_alterado.replace(', \n',', ')
conteudo_alterado = conteudo_alterado.replace('/\n','/')
conteudo_alterado = conteudo_alterado.upper()
conteudo_alterado = conteudo_alterado.replace('FUNDAÇÃO PAULISTANA DE EDUCAÇÃO\nTECNOLOGIA E CULTURA','FUNDAÇÃO PAULISTANA DE EDUCAÇÃO TECNOLOGIA E CULTURA')
conteudo_alterado = conteudo_alterado.replace('HOSPITAL DO SERVIDOR\nPÚBLICO MUNICIPAL','HOSPITAL DO SERVIDOR PÚBLICO MUNICIPAL')
conteudo_alterado = conteudo_alterado.replace('SECRETARIA MUNICIPAL DE ASSIST E DESENV\nSOCIAL','SECRETARIA MUNICIPAL DE ASSIST E DESENV SOCIAL')
conteudo_alterado = conteudo_alterado.replace('SECRETARIA MUNICIPAL DE INFRAESTRUTURA\nURBANA E OB','SECRETARIA MUNICIPAL DE INFRAESTRUTURA URBANA E OB')
conteudo_alterado = conteudo_alterado.replace('SECRETARIA MUNICIPAL DE SEGURANCA\nURBANA','SECRETARIA MUNICIPAL DE SEGURANCA URBANA')
conteudo_alterado = conteudo_alterado.replace('SECRETARIA MUNICIPAL DO VERDE E DO MEIO\nAMBIENTE','SECRETARIA MUNICIPAL DO VERDE E DO MEIO AMBIENTE')
conteudo_alterado = conteudo_alterado.replace('COORDENADORIA DE ADMINISTRAÇÃO\nE FINANÇAS','COORDENADORIA DE ADMINISTRAÇÃO E FINANÇAS')
conteudo_alterado = conteudo_alterado.replace('LICENÇAS MÉDICAS CONCEDIDAS/NEGADAS\nEM GRAU DE RECURSO','LICENÇAS MÉDICAS CONCEDIDAS/NEGADAS EM GRAU DE RECURSO')
conteudo_alterado = conteudo_alterado.replace('RELAÇÃO DE LICENÇAS MÉDICAS NEGADAS\nNOS TERMOS DA LEI 8989/79','RELAÇÃO DE LICENÇAS MÉDICAS NEGADAS NOS TERMOS DA LEI 8989/79')
conteudo_alterado = conteudo_alterado.replace('RELAÇÃO DE FALTAS DOS CONVOCADOS\nPARA AVALIAÇÃO DE ESPECIALISTA','RELAÇÃO DE FALTAS DOS CONVOCADOS PARA AVALIAÇÃO DE ESPECIALISTA')
conteudo_alterado = conteudo_alterado.replace('RELAÇÃO DE FALTAS DOS CONVOCADOS\nPARA JUNTA MÉDICA PARA FINS DE PENSÃO\nPOR MORTE','RELAÇÃO DE FALTAS DOS CONVOCADOS PARA JUNTA MÉDICA PARA FINS DE PENSÃO POR MORTE')
conteudo_alterado = conteudo_alterado.replace('RELAÇÃO DE ALTA DE AT/DT RETIFICADA EM\nFUNÇÃO DE:','RELAÇÃO DE ALTA DE AT/DT RETIFICADA EM FUNÇÃO DE:')
conteudo_alterado = conteudo_alterado.replace('RELAÇÃO DE SERVIDORES SUBMETIDOS À\nAVALIAÇÃO DE ESPECIALISTA\nCOM CAPACIDADE LABORATIVA PARA A SUA\nFUNÇÃO ORIGINAL','RELAÇÃO DE SERVIDORES SUBMETIDOS À AVALIAÇÃO DE ESPECIALISTA COM CAPACIDADE LABORATIVA PARA A SUA FUNÇÃO ORIGINAL')
conteudo_alterado = conteudo_alterado.replace('RELAÇÃO DE CONCESSÃO DE NEXO DE ACIDENTE DO TRABALHO\nDEFERIDO','RELAÇÃO DE CONCESSÃO DE NEXO DE ACIDENTE DO TRABALHO DEFERIDO')
conteudo_alterado = conteudo_alterado.replace('LICENÇAS MÉDICAS DE CURTA DURAÇÃO\nDOC ','LICENÇAS MÉDICAS DE CURTA DURAÇÃO DOC ')
conteudo_alterado = conteudo_alterado.replace('ACI-\nDENTE','ACIDENTE')
conteudo_alterado = conteudo_alterado.replace('CANCELA-\nDAS','CANCELADAS')
conteudo_alterado = conteudo_alterado.replace('RETIFICA-\nDAS','RETIFICADAS')
conteudo_alterado = conteudo_alterado.replace('RECO-\nMENDAÇÃO','RECOMENDAÇÃO')
conteudo_alterado = conteudo_alterado.replace('RECOMENDAÇÃO\nD','RECOMENDAÇÃO D')
conteudo_alterado = conteudo_alterado.replace('LICENÇAS MÉDICAS\nFACE','LICENÇAS MÉDICAS FACE')
conteudo_alterado = conteudo_alterado.replace('LICENÇAS MÉDICAS PARA SERVIDORES SOB REGIME GERAL DA\nPREVIDÊNCIA SOCIAL\nCONCEDIDA DE ACORDO COM O DETERMINADO NA PORTARIA NO\n','LICENÇAS MÉDICAS PARA SERVIDORES SOB REGIME GERAL DA PREVIDÊNCIA SOCIAL CONCEDIDA DE ACORDO COM O DETERMINADO NA PORTARIA NO ')
conteudo_alterado = conteudo_alterado.replace('LICENÇA\nMÉDICA','LICENÇA MÉDICA')
conteudo_alterado = conteudo_alterado.replace('DECRETO NO\n','DECRETO NO')
conteudo_alterado = conteudo_alterado.replace('PUBLICADO\nNO','PUBLICADO NO')
conteudo_alterado = conteudo_alterado.replace('\nCONCEDIDA',' CONCEDIDA')
conteudo_alterado = conteudo_alterado.replace('\nCONCEDIDAS ',' CONCEDIDAS ')
conteudo_alterado = conteudo_alterado.replace(' RETIFICA- DAS ',' RETIFICADAS ')
conteudo_alterado = conteudo_alterado.replace('PORTARIA\n','PORTARIA ')
conteudo_alterado = conteudo_alterado.replace(' TRABALHO\nINDEFERIDO',' TRABALHO INDEFERIDO')
conteudo_alterado = conteudo_alterado.replace(' DOC\nSEI ',' DOC SEI ')
conteudo_alterado = conteudo_alterado.replace(' DA LEI\n',' DA LEI ')
conteudo_alterado = conteudo_alterado.replace('\nAOS ',' AOS ')
conteudo_alterado = conteudo_alterado.replace(' SÃO\n',' SÃO ')
conteudo_alterado = conteudo_alterado.replace(' À\n',' À ')
conteudo_alterado = conteudo_alterado.replace(' ÀS\n',' ÀS ')
conteudo_alterado = conteudo_alterado.replace(' AS\n',' AS ')
conteudo_alterado = conteudo_alterado.replace(' E\n',' E ')
conteudo_alterado = conteudo_alterado.replace(' O\n',' O ')
conteudo_alterado = conteudo_alterado.replace(' OS\n',' OS ')
conteudo_alterado = conteudo_alterado.replace('\nOS ',' OS ')
conteudo_alterado = conteudo_alterado.replace(' DA\n',' DA ')
conteudo_alterado = conteudo_alterado.replace('\nDA ',' DA ')
conteudo_alterado = conteudo_alterado.replace(' DAS\n',' DAS ')
conteudo_alterado = conteudo_alterado.replace('\nDAS ',' DAS ')
conteudo_alterado = conteudo_alterado.replace(' DA(S)\n',' DA(S) ')
conteudo_alterado = conteudo_alterado.replace('\nDA(S) ',' DA(S) ')
conteudo_alterado = conteudo_alterado.replace(' DE\n',' DE ')
conteudo_alterado = conteudo_alterado.replace('\nDE ',' DE ')
conteudo_alterado = conteudo_alterado.replace(' DO\n',' DO ')
conteudo_alterado = conteudo_alterado.replace('\nDO ',' DO ')
conteudo_alterado = conteudo_alterado.replace(' DOS\n',' DOS ')
conteudo_alterado = conteudo_alterado.replace('\nDOS ',' DOS ')
conteudo_alterado = conteudo_alterado.replace(' DO(S)\n',' DO(S) ')
conteudo_alterado = conteudo_alterado.replace('\nDO(S) ',' DO(S) ')
conteudo_alterado = conteudo_alterado.replace(' COM\n',' COM ')
conteudo_alterado = conteudo_alterado.replace('\nCOM ',' COM ')
conteudo_alterado = conteudo_alterado.replace(' EM\n',' EM ')
conteudo_alterado = conteudo_alterado.replace('\nEM ',' EM ')
conteudo_alterado = conteudo_alterado.replace(' PARA\n',' PARA ')
conteudo_alterado = conteudo_alterado.replace('\nPARA ',' PARA ')
conteudo_alterado = conteudo_alterado.replace(' POR\n',' POR ')
conteudo_alterado = conteudo_alterado.replace(' NA\n',' NA ')
conteudo_alterado = conteudo_alterado.replace(' NAS\n',' NAS ')
conteudo_alterado = conteudo_alterado.replace('\nNAS ',' NAS ')
conteudo_alterado = conteudo_alterado.replace(' NO\n',' NO ')
conteudo_alterado = conteudo_alterado.replace(' NOS\n',' NOS ')
conteudo_alterado = conteudo_alterado.replace('\nNOS ',' NOS ')
conteudo_alterado = conteudo_alterado.replace('A PARTIR DE',' A_PARTIR_DE')
conteudo_alterado = conteudo_alterado.replace('RF NOME DIAS A PARTIR DE ARTIGO','RF NOME DIAS A_PARTIR_DE ARTIGO')
conteudo_alterado = conteudo_alterado.replace('RF NOME A_PARTIR_DE ','RF NOME A_PARTIR_DE\n')
conteudo_alterado = conteudo_alterado.replace('REG. FUNC. NOME CONTROLE NO ','RF NOME CONTROLE_NO\n')
conteudo_alterado = conteudo_alterado.replace('R.F./V NOME DIAS/EXERCÍCIO A_PARTIR_DE','RF/V NOME DIAS/EXERCÍCIO A_PARTIR_DE')
conteudo_alterado = conteudo_alterado.replace('REG. FUNC/VINC NOME NÍVEL CAT. SÍMBOLO A_PARTIR_DE','RF VINC NOME NÍVEL_CAT SÍMBOLO A_PARTIR_DE')
conteudo_alterado = conteudo_alterado.replace('RF V NOME EH A PARTIR MOTIVO','RF VINC NOME EH A_PARTIR MOTIVO')

In [217]:
conteudo_alterado = conteudo_alterado.replace('\t',' ')
conteudo_alterado = conteudo_alterado.replace('""','')
conteudo_alterado = conteudo_alterado.replace(' \n','\n')
conteudo_alterado = conteudo_alterado.replace('\n ','\n')
conteudo_alterado = conteudo_alterado.replace('    ',' ')
conteudo_alterado = conteudo_alterado.replace('   ',' ')
conteudo_alterado = conteudo_alterado.replace('  ',' ')
conteudo_alterado = conteudo_alterado.replace('-\n','')
conteudo_alterado = conteudo_alterado.replace(',\n',', ')
conteudo_alterado = conteudo_alterado.replace(', \n',', ')
conteudo_alterado = conteudo_alterado.replace('/\n','/')
conteudo_alterado = conteudo_alterado.upper()
conteudo_alterado = conteudo_alterado.replace('FUNDAÇÃO PAULISTANA DE EDUCAÇÃO\nTECNOLOGIA E CULTURA','FUNDAÇÃO PAULISTANA DE EDUCAÇÃO TECNOLOGIA E CULTURA')
conteudo_alterado = conteudo_alterado.replace('HOSPITAL DO SERVIDOR\nPÚBLICO MUNICIPAL','HOSPITAL DO SERVIDOR PÚBLICO MUNICIPAL')
conteudo_alterado = conteudo_alterado.replace('SECRETARIA MUNICIPAL DE ASSIST E DESENV\nSOCIAL','SECRETARIA MUNICIPAL DE ASSIST E DESENV SOCIAL')
conteudo_alterado = conteudo_alterado.replace('SECRETARIA MUNICIPAL DE INFRAESTRUTURA\nURBANA E OB','SECRETARIA MUNICIPAL DE INFRAESTRUTURA URBANA E OB')
conteudo_alterado = conteudo_alterado.replace('SECRETARIA MUNICIPAL DE SEGURANCA\nURBANA','SECRETARIA MUNICIPAL DE SEGURANCA URBANA')
conteudo_alterado = conteudo_alterado.replace('SECRETARIA MUNICIPAL DO VERDE E DO MEIO\nAMBIENTE','SECRETARIA MUNICIPAL DO VERDE E DO MEIO AMBIENTE')
conteudo_alterado = conteudo_alterado.replace('COORDENADORIA DE ADMINISTRAÇÃO\nE FINANÇAS','COORDENADORIA DE ADMINISTRAÇÃO E FINANÇAS')
conteudo_alterado = conteudo_alterado.replace('LICENÇAS MÉDICAS CONCEDIDAS/NEGADAS\nEM GRAU DE RECURSO','LICENÇAS MÉDICAS CONCEDIDAS/NEGADAS EM GRAU DE RECURSO')
conteudo_alterado = conteudo_alterado.replace('RELAÇÃO DE LICENÇAS MÉDICAS NEGADAS\nNOS TERMOS DA LEI 8989/79','RELAÇÃO DE LICENÇAS MÉDICAS NEGADAS NOS TERMOS DA LEI 8989/79')
conteudo_alterado = conteudo_alterado.replace('RELAÇÃO DE FALTAS DOS CONVOCADOS\nPARA AVALIAÇÃO DE ESPECIALISTA','RELAÇÃO DE FALTAS DOS CONVOCADOS PARA AVALIAÇÃO DE ESPECIALISTA')
conteudo_alterado = conteudo_alterado.replace('RELAÇÃO DE FALTAS DOS CONVOCADOS\nPARA JUNTA MÉDICA PARA FINS DE PENSÃO\nPOR MORTE','RELAÇÃO DE FALTAS DOS CONVOCADOS PARA JUNTA MÉDICA PARA FINS DE PENSÃO POR MORTE')
conteudo_alterado = conteudo_alterado.replace('RELAÇÃO DE ALTA DE AT/DT RETIFICADA EM\nFUNÇÃO DE:','RELAÇÃO DE ALTA DE AT/DT RETIFICADA EM FUNÇÃO DE:')
conteudo_alterado = conteudo_alterado.replace('RELAÇÃO DE SERVIDORES SUBMETIDOS À\nAVALIAÇÃO DE ESPECIALISTA\nCOM CAPACIDADE LABORATIVA PARA A SUA\nFUNÇÃO ORIGINAL','RELAÇÃO DE SERVIDORES SUBMETIDOS À AVALIAÇÃO DE ESPECIALISTA COM CAPACIDADE LABORATIVA PARA A SUA FUNÇÃO ORIGINAL')
conteudo_alterado = conteudo_alterado.replace('RELAÇÃO DE CONCESSÃO DE NEXO DE ACIDENTE DO TRABALHO\nDEFERIDO','RELAÇÃO DE CONCESSÃO DE NEXO DE ACIDENTE DO TRABALHO DEFERIDO')
conteudo_alterado = conteudo_alterado.replace('LICENÇAS MÉDICAS DE CURTA DURAÇÃO\nDOC ','LICENÇAS MÉDICAS DE CURTA DURAÇÃO DOC ')
conteudo_alterado = conteudo_alterado.replace('ACI-\nDENTE','ACIDENTE')
conteudo_alterado = conteudo_alterado.replace('CANCELA-\nDAS','CANCELADAS')
conteudo_alterado = conteudo_alterado.replace('RETIFICA-\nDAS','RETIFICADAS')
conteudo_alterado = conteudo_alterado.replace('RECO-\nMENDAÇÃO','RECOMENDAÇÃO')
conteudo_alterado = conteudo_alterado.replace('RECOMENDAÇÃO\nD','RECOMENDAÇÃO D')
conteudo_alterado = conteudo_alterado.replace('LICENÇAS MÉDICAS\nFACE','LICENÇAS MÉDICAS FACE')
conteudo_alterado = conteudo_alterado.replace('LICENÇAS MÉDICAS PARA SERVIDORES SOB REGIME GERAL DA\nPREVIDÊNCIA SOCIAL\nCONCEDIDA DE ACORDO COM O DETERMINADO NA PORTARIA NO\n','LICENÇAS MÉDICAS PARA SERVIDORES SOB REGIME GERAL DA PREVIDÊNCIA SOCIAL CONCEDIDA DE ACORDO COM O DETERMINADO NA PORTARIA NO ')
conteudo_alterado = conteudo_alterado.replace('LICENÇA\nMÉDICA','LICENÇA MÉDICA')
conteudo_alterado = conteudo_alterado.replace('DECRETO NO\n','DECRETO NO')
conteudo_alterado = conteudo_alterado.replace('PUBLICADO\nNO','PUBLICADO NO')
conteudo_alterado = conteudo_alterado.replace('\nCONCEDIDA',' CONCEDIDA')
conteudo_alterado = conteudo_alterado.replace('\nCONCEDIDAS ',' CONCEDIDAS ')
conteudo_alterado = conteudo_alterado.replace(' RETIFICA- DAS ',' RETIFICADAS ')
conteudo_alterado = conteudo_alterado.replace('PORTARIA\n','PORTARIA ')
conteudo_alterado = conteudo_alterado.replace(' TRABALHO\nINDEFERIDO',' TRABALHO INDEFERIDO')
conteudo_alterado = conteudo_alterado.replace(' DOC\nSEI ',' DOC SEI ')
conteudo_alterado = conteudo_alterado.replace(' DA LEI\n',' DA LEI ')
conteudo_alterado = conteudo_alterado.replace('\nAOS ',' AOS ')
conteudo_alterado = conteudo_alterado.replace(' SÃO\n',' SÃO ')
conteudo_alterado = conteudo_alterado.replace(' À\n',' À ')
conteudo_alterado = conteudo_alterado.replace(' ÀS\n',' ÀS ')
conteudo_alterado = conteudo_alterado.replace(' AS\n',' AS ')
conteudo_alterado = conteudo_alterado.replace(' E\n',' E ')
conteudo_alterado = conteudo_alterado.replace(' O\n',' O ')
conteudo_alterado = conteudo_alterado.replace(' OS\n',' OS ')
conteudo_alterado = conteudo_alterado.replace('\nOS ',' OS ')
conteudo_alterado = conteudo_alterado.replace(' DA\n',' DA ')
conteudo_alterado = conteudo_alterado.replace('\nDA ',' DA ')
conteudo_alterado = conteudo_alterado.replace(' DAS\n',' DAS ')
conteudo_alterado = conteudo_alterado.replace('\nDAS ',' DAS ')
conteudo_alterado = conteudo_alterado.replace(' DA(S)\n',' DA(S) ')
conteudo_alterado = conteudo_alterado.replace('\nDA(S) ',' DA(S) ')
conteudo_alterado = conteudo_alterado.replace(' DE\n',' DE ')
conteudo_alterado = conteudo_alterado.replace('\nDE ',' DE ')
conteudo_alterado = conteudo_alterado.replace(' DO\n',' DO ')
conteudo_alterado = conteudo_alterado.replace('\nDO ',' DO ')
conteudo_alterado = conteudo_alterado.replace(' DOS\n',' DOS ')
conteudo_alterado = conteudo_alterado.replace('\nDOS ',' DOS ')
conteudo_alterado = conteudo_alterado.replace(' DO(S)\n',' DO(S) ')
conteudo_alterado = conteudo_alterado.replace('\nDO(S) ',' DO(S) ')
conteudo_alterado = conteudo_alterado.replace(' COM\n',' COM ')
conteudo_alterado = conteudo_alterado.replace('\nCOM ',' COM ')
conteudo_alterado = conteudo_alterado.replace(' EM\n',' EM ')
conteudo_alterado = conteudo_alterado.replace('\nEM ',' EM ')
conteudo_alterado = conteudo_alterado.replace(' PARA\n',' PARA ')
conteudo_alterado = conteudo_alterado.replace('\nPARA ',' PARA ')
conteudo_alterado = conteudo_alterado.replace(' POR\n',' POR ')
conteudo_alterado = conteudo_alterado.replace(' NA\n',' NA ')
conteudo_alterado = conteudo_alterado.replace(' NAS\n',' NAS ')
conteudo_alterado = conteudo_alterado.replace('\nNAS ',' NAS ')
conteudo_alterado = conteudo_alterado.replace(' NO\n',' NO ')
conteudo_alterado = conteudo_alterado.replace(' NOS\n',' NOS ')
conteudo_alterado = conteudo_alterado.replace('\nNOS ',' NOS ')
conteudo_alterado = conteudo_alterado.replace('A PARTIR DE',' A_PARTIR_DE')
conteudo_alterado = conteudo_alterado.replace('RF NOME DIAS A PARTIR DE ARTIGO','RF NOME DIAS A_PARTIR_DE ARTIGO')
conteudo_alterado = conteudo_alterado.replace('RF NOME A_PARTIR_DE ','RF NOME A_PARTIR_DE\n')
conteudo_alterado = conteudo_alterado.replace('REG. FUNC. NOME CONTROLE NO ','RF NOME CONTROLE_NO\n')
conteudo_alterado = conteudo_alterado.replace('R.F./V NOME DIAS/EXERCÍCIO A_PARTIR_DE','RF/V NOME DIAS/EXERCÍCIO A_PARTIR_DE')
conteudo_alterado = conteudo_alterado.replace('REG. FUNC/VINC NOME NÍVEL CAT. SÍMBOLO A_PARTIR_DE','RF VINC NOME NÍVEL_CAT SÍMBOLO A_PARTIR_DE')
conteudo_alterado = conteudo_alterado.replace('RF V NOME EH A PARTIR MOTIVO','RF VINC NOME EH A_PARTIR MOTIVO')

In [218]:
# Localiza o início das seções SERVIDORES e CONCURSOS
sc_servidores = conteudo_alterado.find('\nSERVIDORES\n')
sc_concursos = conteudo_alterado.find('\nCONCURSOS\n')
print(f"SERVIRODRES: {sc_servidores} / CONCURSOS: {sc_concursos}")
conteudo_servidores = conteudo_alterado[sc_servidores:sc_concursos]
#conteudo_servidores = conteudo

SERVIRODRES: 943805 / CONCURSOS: 1290155


In [224]:
#print(conteudo_servidores)
save_file(SAVE_DIR,conteudo_servidores,'SECAO_SERVIDORES')

'c:\\Projetos\\scrapy_dom\\outputs_files\\2022-05-31\\SECAO_SERVIDORES.txt'

In [225]:
#print(conteudo_servidores)

### Relação de Licença Médica

#### CONTEÚDO PRINCIPAL

In [226]:
conteudo_licenca_medica = conteudo_servidores

In [281]:
i = 0
#rl_lm =  conteudo_licenca_medica.split('\n')
rl_lm =  conteudo_licenca_medica.split('\n')
licencas = OrderedDict({})
listas = []

for line in rl_lm:
    if 'LICENÇAS MÉDICAS' in line and not 'SEÇÃO DE LICENÇAS MÉDICAS' in line:
        licencas.update({
            line : i
        })
        listas.append(i)
    elif ('RELAÇÃO DE' in line or 'RELAÇÃO DOS' in line) or 'RELAÇÃO DE LICENÇA MÉDICA' in line:
        licencas.update({
            line : i
        })
        listas.append(i)
    elif 'SERVIDORES COM ' in line:
        licencas.update({
            line : i
        })
    i += 1

print(licencas)
#print(listas)
print(f'\nTotal de listas: {len(listas)}')

OrderedDict([('A DIVISÃO DE RECURSOS HUMANOS DA SECRETÁRIA MUNICIPAL DE SEGURANÇA URBANA, CONFORME ANEXO I A QUE SE REFERE O ART. 2O DA PORTARIA 097/SEMPLA/2012 DIVULGA A RELAÇÃO DOS SERVIDORES PARA CESSAÇÃO DO HORÁRIO DE ESTUDANTE E AUSÊNCIA NOS DIAS DE REALIZAÇÃO DE PROVAS, CONFORME SEGUE:', 373), ('A DIVISÃO DE RECURSOS HUMANOS DA SECRETARIA MUNICIPAL DE SEGURANÇA URBANA DIVULGA A RELAÇÃO DOS SERVIDORES PARA CONCESSÃO DO HORÁRIO DE ESTUDANTE E AUSÊNCIA NOS DIAS DE REALIZAÇÃO DE PROVAS POR ATENDER AOS REQUISITOS EXIGIDOS NO DECRETO 58.073/18, CONFORME SEGUE:', 377), ('RELAÇÃO DE SERVIDORES CONVOCADOS PARA PERÍCIA DE LICENÇA MÉDICA PRÓPRIO SERVIDOR - LONGA DURAÇÃO', 452), ('RELAÇÃO DE SERVIDORES CONVOCADOS PARA AVALIAÇÃO MÉDICO', 526), ('RELAÇÃO DE SERVIDORES CONVOCADOS PARA AVALIAÇÃO COM MÉDICO PERITO ESPECIALISTA', 553), ('RELAÇÃO DE CONVOCADOS PARA PERÍCIA MÉDICA DE ISENÇÃO DE IMPOSTO DE RENDA', 569), ('RELAÇÃO DE CONVOCADOS PARA JUNTA DE AVALIAÇÃO DE INCAPACIDADE PERMANENTE PARA O

In [282]:
lista_licencas = [
    'RELAÇÃO DE LICENÇAS MÉDICAS NOS TERMOS DA LEI 8989/79',
    'RELAÇÃO DE LICENÇAS MÉDICAS NEGADAS NOS TERMOS DA LEI 8989/79',
    'RELAÇÃO DE FALTAS DOS CONVOCADOS PARA JUNTA MÉDICA PARA FINS DE PENSÃO POR MORTE',
    'RELAÇÃO DE FALTA DO SERVIDOR À PERÍCIA',
    'LICENÇAS MÉDICAS CONCEDIDAS/NEGADAS EM GRAU DE RECURSO',
    'RELAÇÃO DE FALTAS DOS CONVOCADOS PARA AVALIAÇÃO DE ESPECIALISTA',
    'RELAÇÃO DE FALTAS DOS CONVOCADOS P/ AVALIAÇÃO DE ISENÇÃO DE IMPOSTO DE RENDA',
    'RELAÇÃO DE FALTA DO SERVIDOR À PERÍCIA',
    'RELAÇÃO DE SERVIDORES SUBMETIDOS À AVALIAÇÃO DE CAPACIDADE LABORATIVA COM CAPACIDADE LABORATIVA PARA A SUA FUNÇÃO',
    'ENGLOBAMENTO DE LICENÇAS MÉDICAS FACE PORTARIA 11/84',
    'RELAÇÃO DE ALTAS DO ACIDENTE DO TRABALHO E DA DOENÇA DO TRABALHO',
    'RELAÇÃO DE CONCESSÃO DE NEXO DE ACIDENTE DO TRABALHO DEFERIDO',
    'RELAÇÃO DE CONCESSÃO DE NEXO DE ACIDENTE DO TRABALHO DEFERIDO EM GRAU DE RECURSO',
    'RELAÇÃO DE CONCESSÃO DE NEXO DE ACIDENTE DO TRABALHO INDEFERIDO',
]

In [256]:
id = 0
str_licencas = 'ID;LISTAGEM;INICIO;MAPEADA;QTD\n'

for relacao, linha in licencas.items():
    id += 1
    mapeada = relacao in lista_licencas
    if id == 1:
        str_licencas += f'{id};{relacao};{linha};{mapeada};'
        lin_anterior = linha
    if id > 1:
        qtd = linha - lin_anterior
        str_licencas += f'{qtd}\n{id};{relacao};{linha};{mapeada};'
        lin_anterior = lin_anterior

#print(str_licencas)
save_file(SAVE_DIR,str_licencas,'00-LISTAGENS_DOM')

'c:\\Projetos\\scrapy_dom\\outputs_files\\2022-05-31\\00-LISTAGENS_DOM.txt'

#### RELAÇÃO DE LICENÇAS MÉDICAS NOS TERMOS DA LEI 8989/79

In [223]:
RELACAO = 'RELAÇÃO DE LICENÇAS MÉDICAS NOS TERMOS DA LEI 8989/79'

if RELACAO in licencas:
    list_items = rl_lm[licencas[RELACAO]:listas[listas.index(licencas[RELACAO])+1]]
else:
    list_items = []
list_items

['RELAÇÃO DE LICENÇAS MÉDICAS NOS TERMOS DA LEI 8989/79',
 'SECRETARIA MUNICIPAL DA FAZENDA',
 'RF NOME DIAS A_PARTIR_DE ARTIGO',
 '6740090/1 CEZAR AUGUSTO MACHADO SIPOLI 090 07/04/2022 143',
 'SECRETARIA MUNICIPAL DA SAUDE',
 'RF NOME DIAS A_PARTIR_DE ARTIGO',
 '1380966/5 ROMEU CESAR DE JESUS 044 02/04/2022 143',
 '1384139/2 MARIA DO PERPETUO SOCORRO LIMA CARVALHO 010 21/10/2021 143',
 '5490081/2 IARA MARIA FERREIRA 009 26/10/2021 143',
 '5560705/4 DULCINEA APARECIDA DA SILVA ARAUJO 025 30/03/2022 143',
 '5580994/3 DORA REGINA ALMEIDA JECKS SPAGNOL 006 21/12/2021 143',
 '5800960/3 LUIZ CARLOS COLACIQUE 008 27/09/2021 143',
 '5832179/1 ZILDA SANDRA DE JESUS ANDRE 005 29/11/2021 143',
 '5914183/3 MARIA DE JESUS RAMOS 087 01/04/2022 143',
 '6043836/3 ESTER LUCCAS RAMOS 004 03/09/2021 143',
 '6108709/1 LEONARDO CARDEAL ARAUJO 007 29/01/2022 143',
 '6203566/1 LAURO JAMES DA SILVA MORAES 092 30/08/2021 143',
 '6219390/1 ALTAIR LAURINDO DE SOUZA 060 03/04/2022 143',
 '6302441/4 MARIA DAS GRA

In [113]:
'''Listagem: RELAÇÃO DE LICENÇAS MÉDICAS NOS TERMOS DA LEI 8989/79
    COLUNAS:
        1. RF           : numeric(7) s_item[0][0]
        2. VINC         : numeric(1) s_item[0][1]
        3. NOME         : str()      s_item[2:-3]
        4. DIAS         : numeric()  s_item[-3]
        5. A_PARTIR_DE  : date()     s_item[-2]
        6. ARTIGO       : str()      s_item[-1]
'''

i = 0
SECRETARIA = ''
VALIDA_LINHA = ''
str_list_items = 'RF;VINC;NOME;DIAS;A_PARTIR_DE;ARTIGO;SECRETARIA;LISTA;DOM\n'

for item in list_items:
    s_item = item.strip().split(' ')
    #print(f'analise: {s_item}')
    if 'RF NOME DIAS A_PARTIR_DE ARTIGO' in item:
        SECRETARIA = list_items[i-1]
    elif len(s_item) >= 5:
        if len(s_item) >= 5 and s_item[0][-1].isnumeric() and s_item[-3][0].isnumeric() and s_item[-2][0].isnumeric():
            rf = s_item[0].split('/')[0]
            vinc = s_item[0].split('/')[1]
            nome = str(' ').join(s_item[1:-3])
            str_list_items += f'{rf};{vinc};{nome};{s_item[-3]};{s_item[-2]};{s_item[-1]};{SECRETARIA};{RELACAO};{DOM}\n'
            #print(str_list_items)
    elif len(s_item) >= 2 and s_item[0][-1].isnumeric() and VALIDA_LINHA == False:
        linha = s_item
        VALIDA_LINHA = True
    elif VALIDA_LINHA == True:
        linha += f' {item}'
        s_item = item.split(' ')
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-3])
        str_list_items += f'{rf};{vinc};{nome};{s_item[-3]};{s_item[-2]};{s_item[-1]};{SECRETARIA};{RELACAO};{DOM}\n'
        print(str_list_items)
        VALIDA_LINHA = False
    i += 1
    #print(i)

In [114]:
RELACAO = str('_').join(RELACAO.replace('/','_').split(' '))
save_file(SAVE_DIR,str_list_items,RELACAO,'csv')

'c:\\Projetos\\scrapy_dom\\outputs_files\\2022-05-31\\RELAÇÃO_DE_LICENÇAS_MÉDICAS_NOS_TERMOS_DA_LEI_8989_79.csv'

#### RELAÇÃO DE LICENÇAS MÉDICAS NEGADAS NOS TERMOS DA LEI 8989/79

In [115]:
RELACAO = 'RELAÇÃO DE LICENÇAS MÉDICAS NEGADAS NOS TERMOS DA LEI 8989/79'

if RELACAO in licencas:
    list_items = rl_lm[licencas[RELACAO]:listas[listas.index(licencas[RELACAO])+1]]
else:
    list_items = []
#list_items


In [116]:
'''
    COLUNAS:
        1. RF           : numeric(7) s_item[0][0]
        2. VINC         : numeric(1) s_item[0][1]
        3. NOME         : str()      s_item[1:-2]
        4. DIAS         : null
        5. A_PARTIR_DE  : date()     s_item[-2]
        6. ARTIGO       : str()      s_item[-1]
'''

i = 0
SECRETARIA = ''
VALIDA_LINHA = ''
str_list_items = 'RF;VINC;NOME;DIAS;A_PARTIR_DE;ARTIGO;SECRETARIA;LISTA;DOM\n'

for item in list_items:
    s_item = item.strip().split(' ')
    if 'RF NOME DIAS A_PARTIR_DE ARTIGO' in item:
        SECRETARIA = list_items[i-1]
    elif len(s_item) >= 5 and s_item[0][-1].isnumeric() and s_item[-2][0].isnumeric():
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-2])
        str_list_items += f'{rf};{vinc};{nome};;{s_item[-2]};{s_item[-1]};{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
    elif s_item[0][-1].isnumeric() and VALIDA_LINHA == False:
        linha = s_item
        VALIDA_LINHA = True
    elif VALIDA_LINHA == True:
        linha += f' {item}'
        s_item = item.split(' ')
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-2])
        str_list_items += f'{rf};{vinc};{nome};;{s_item[-2]};{s_item[-1]};{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
        VALIDA_LINHA = False
    i += 1

In [117]:
RELACAO = str('_').join(RELACAO.replace('/','_').split(' '))
save_file(SAVE_DIR,str_list_items,RELACAO,'csv')

'c:\\Projetos\\scrapy_dom\\outputs_files\\2022-05-31\\RELAÇÃO_DE_LICENÇAS_MÉDICAS_NEGADAS_NOS_TERMOS_DA_LEI_8989_79.csv'

#### RELAÇÃO DE FALTAS DOS CONVOCADOS PARA JUNTA MÉDICA PARA FINS DE PENSÃO POR MORTE

In [118]:
RELACAO = 'RELAÇÃO DE FALTAS DOS CONVOCADOS PARA JUNTA MÉDICA PARA FINS DE PENSÃO POR MORTE'

if RELACAO in licencas:
    list_items = rl_lm[licencas[RELACAO]:listas[listas.index(licencas[RELACAO])+1]]
else:
    list_items = []
#list_items

In [119]:
'''
    COLUNAS:
        1. RF           : numeric(7) s_item[0][0]
        2. VINC         : numeric(1) s_item[0][1]
        3. NOME         : str()      s_item[1:-1]
        4. DIAS         : null
        5. A_PARTIR_DE  : date()     s_item[-1]
        6. ARTIGO       : null
'''

i = 0
SECRETARIA = ''
VALIDA_LINHA = ''
str_list_items = 'RF;VINC;NOME;DIAS;A_PARTIR_DE;ARTIGO;SECRETARIA;LISTA;DOM\n'

for item in list_items:
    s_item = item.strip().split(' ')
    if 'RF NOME DIAS A_PARTIR_DE ARTIGO' in item:
        SECRETARIA = list_items[i-1]
    elif len(s_item) >= 5 and s_item[0][0].isnumeric() and s_item[-1][0].isnumeric():
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-1])
        str_list_items += f'{rf};{vinc};{nome};;{s_item[-1]};;{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
    elif s_item[0][-1].isnumeric() and VALIDA_LINHA == False:
        linha = s_item
        VALIDA_LINHA = True
    elif VALIDA_LINHA == True:
        linha += f' {item}'
        s_item = item.split(' ')
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-1])
        str_list_items += f'{rf};{vinc};{nome};;{s_item[-1]};;{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
        VALIDA_LINHA = False
    i += 1

In [120]:
RELACAO = str('_').join(RELACAO.replace('/','_').split(' '))
save_file(SAVE_DIR,str_list_items,RELACAO,'csv')

'c:\\Projetos\\scrapy_dom\\outputs_files\\2022-05-31\\RELAÇÃO_DE_FALTAS_DOS_CONVOCADOS_PARA_JUNTA_MÉDICA_PARA_FINS_DE_PENSÃO_POR_MORTE.csv'

#### RELAÇÃO DE FALTA DO SERVIDOR À PERÍCIA

In [121]:
RELACAO = 'RELAÇÃO DE FALTA DO SERVIDOR À PERÍCIA'

if RELACAO in licencas:
    list_items = rl_lm[licencas[RELACAO]:listas[listas.index(licencas[RELACAO])+1]]
else:
    list_items = []
#list_items

In [122]:
i = 0
SECRETARIA = ''
str_list_items = 'RF;NOME;DIAS;A_PARTIR_DE;ARTIGO;SECRETARIA;LISTA;DOM\n'

for item in list_items:
    s_item = item.strip().split(' ')
    if 'RF NOME DIAS A_PARTIR_DE ARTIGO' in item:
        SECRETARIA = list_items[i-1]
    elif len(s_item) >= 4 and s_item[0][0].isnumeric():
        if s_item[-3].isnumeric():
            nome = str(' ').join(s_item[1:-3])
            str_list_items += f'{s_item[0]};{nome};{s_item[-3]};{s_item[-2]};{s_item[-1]};{SECRETARIA};{RELACAO}/{DOM}\n'
            #print(str_list_items)
        else:
            nome = str(' ').join(s_item[1:-2])
            str_list_items += f'{s_item[0]};{nome};;{s_item[-2]};{s_item[-1]};{SECRETARIA};{RELACAO};{DOM}\n'
            #print(str_list_items)
    i += 1

In [123]:
RELACAO = str('_').join(RELACAO.replace('/','_').split(' '))
save_file(SAVE_DIR,str_list_items,RELACAO,'csv')

'c:\\Projetos\\scrapy_dom\\outputs_files\\2022-05-31\\RELAÇÃO_DE_FALTA_DO_SERVIDOR_À_PERÍCIA.csv'

#### LICENÇAS MÉDICAS CONCEDIDAS/NEGADAS EM GRAU DE RECURSO

In [124]:
RELACAO = 'LICENÇAS MÉDICAS CONCEDIDAS/NEGADAS EM GRAU DE RECURSO'

if RELACAO in licencas:
    list_items = rl_lm[licencas[RELACAO]:listas[listas.index(licencas[RELACAO])+1]]
else:
    list_items = []
list_items

[]

In [125]:
'''Listagem: LICENÇAS MÉDICAS CONCEDIDAS/NEGADAS EM GRAU DE RECURSO
    COLUNAS:
        1. RF           : numeric(7) s_item[0][0]
        2. VINC         : numeric(1) s_item[0][1]
        3. NOME         : str()      s_item[2:-3]
        4. DIAS         : numeric()  s_item[-3]
        5. A_PARTIR_DE  : date()     s_item[-2]
        6. ARTIGO       : str()      s_item[-1]
'''

i = 0
SECRETARIA = ''
VALIDA_LINHA = ''
str_list_items = 'RF;VINC;NOME;DIAS;A_PARTIR_DE;ARTIGO;SECRETARIA;LISTA;DOM\n'

for item in list_items:
    s_item = item.strip().split(' ')
    if 'RF NOME DIAS A_PARTIR_DE ARTIGO' in item:
        SECRETARIA = list_items[i-1]
    elif len(s_item) >= 5 and s_item[0][-1].isnumeric() and s_item[-3][0].isnumeric() and s_item[-2][0].isnumeric():
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-3])
        str_list_items += f'{rf};{vinc};{nome};{s_item[-3]};{s_item[-2]};{s_item[-1]};{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
    elif s_item[0][-1].isnumeric() and VALIDA_LINHA == False:
        linha = s_item
        VALIDA_LINHA = True
    elif VALIDA_LINHA == True:
        linha += f' {item}'
        s_item = item.split(' ')
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-3])
        str_list_items += f'{rf};{vinc};{nome};{s_item[-3]};{s_item[-2]};{s_item[-1]};{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
        VALIDA_LINHA = False
    i += 1

In [126]:
RELACAO = str('_').join(RELACAO.replace('/','_').split(' '))
save_file(SAVE_DIR,str_list_items,RELACAO,'csv')

'c:\\Projetos\\scrapy_dom\\outputs_files\\2022-05-31\\LICENÇAS_MÉDICAS_CONCEDIDAS_NEGADAS_EM_GRAU_DE_RECURSO.csv'

#### RELAÇÃO DE FALTAS DOS CONVOCADOS PARA AVALIAÇÃO DE ESPECIALISTA

In [127]:
RELACAO = 'RELAÇÃO DE FALTAS DOS CONVOCADOS PARA AVALIAÇÃO DE ESPECIALISTA'

if RELACAO in licencas:
    list_items = rl_lm[licencas[RELACAO]:listas[listas.index(licencas[RELACAO])+1]]
else:
    list_items = []
list_items

[]

In [128]:
'''
    COLUNAS:
        1. RF           : numeric(7) s_item[0][0]
        2. VINC         : numeric(1) s_item[0][1]
        3. NOME         : str()      s_item[1:-1]
        4. A_PARTIR_DE  : date()     s_item[-1]
'''

i = 0
SECRETARIA = ''
VALIDA_LINHA = ''
str_list_items = 'RF;VINC;NOME;DATA;SECRETARIA;LISTA;DOM\n'

for item in list_items:
    s_item = item.strip().split(' ')
    if 'RF NOME DIAS A_PARTIR_DE ARTIGO' in item:
        SECRETARIA = list_items[i-1]
    elif len(s_item) >= 4 and s_item[0][0].isnumeric() and s_item[-1][0].isnumeric():
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-1])
        str_list_items += f'{rf};{vinc};{nome};{s_item[-1]};{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
    elif s_item[0][-1].isnumeric() and VALIDA_LINHA == False:
        linha = s_item
        VALIDA_LINHA = True
    elif VALIDA_LINHA == True:
        linha += f' {item}'
        s_item = item.split(' ')
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-1])
        str_list_items += f'{rf};{vinc};{nome};{s_item[-1]};{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
        VALIDA_LINHA = False
    i += 1

In [129]:
RELACAO = str('_').join(RELACAO.replace('/','_').split(' '))
save_file(SAVE_DIR,str_list_items,RELACAO,'csv')

'c:\\Projetos\\scrapy_dom\\outputs_files\\2022-05-31\\RELAÇÃO_DE_FALTAS_DOS_CONVOCADOS_PARA_AVALIAÇÃO_DE_ESPECIALISTA.csv'

#### RELAÇÃO DE FALTAS DOS CONVOCADOS P/ AVALIAÇÃO DE ISENÇÃO DE IMPOSTO DE RENDA

In [130]:
RELACAO = 'RELAÇÃO DE FALTAS DOS CONVOCADOS P/ AVALIAÇÃO DE ISENÇÃO DE IMPOSTO DE RENDA'

if RELACAO in licencas:
    list_items = rl_lm[licencas[RELACAO]:listas[listas.index(licencas[RELACAO])+1]]
else:
    list_items = []
#list_items

In [131]:
'''
    COLUNAS:
        1. RF           : numeric(7) s_item[0][0]
        2. VINC         : numeric(1) s_item[0][1]
        3. NOME         : str()      s_item[1:-1]
        4. A_PARTIR_DE  : date()     s_item[-1]
'''

i = 0
SECRETARIA = ''
VALIDA_LINHA = ''
str_list_items = 'RF;VINC;NOME;A_PARITR_DE;SECRETARIA;LISTA;DOM\n'

for item in list_items:
    s_item = item.strip().split(' ')
    if 'RF NOME DIAS A_PARTIR_DE ARTIGO' in item:
        SECRETARIA = list_items[i-1]
    elif len(s_item) >= 4 and s_item[0][0].isnumeric() and s_item[-1][0].isnumeric():
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-1])
        str_list_items += f'{rf};{vinc};{nome};{s_item[-1]};{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
    elif s_item[0][-1].isnumeric() and VALIDA_LINHA == False:
        linha = s_item
        VALIDA_LINHA = True
    elif VALIDA_LINHA == True:
        linha += f' {item}'
        s_item = item.split(' ')
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-1])
        str_list_items += f'{rf};{vinc};{nome};{s_item[-1]};{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
        VALIDA_LINHA = False
    i += 1

In [132]:
RELACAO = str('_').join(RELACAO.replace('/','_').split(' '))
save_file(SAVE_DIR,str_list_items,RELACAO,'csv')

'c:\\Projetos\\scrapy_dom\\outputs_files\\2022-05-31\\RELAÇÃO_DE_FALTAS_DOS_CONVOCADOS_P__AVALIAÇÃO_DE_ISENÇÃO_DE_IMPOSTO_DE_RENDA.csv'

#### RELAÇÃO DE FALTA DO SERVIDOR À PERÍCIA

In [133]:
RELACAO = 'RELAÇÃO DE FALTA DO SERVIDOR À PERÍCIA'

if RELACAO in licencas:
    list_items = rl_lm[licencas[RELACAO]:listas[listas.index(licencas[RELACAO])+1]]
else:
    list_items = []
#list_items

In [134]:
'''
    COLUNAS:
        1. RF           : numeric(7) s_item[0][0]
        2. VINC         : numeric(1) s_item[0][1]
        3. NOME         : str()      s_item[1:-2]
        4. A_PARTIR_DE  : date()     s_item[-2]
        5. ARTIGO       : str()      s_item[-1]
'''
    
i = 0
SECRETARIA = ''
VALIDA_LINHA = ''
str_list_items = 'RF;VINC;NOME;DIAS;A_PARTIR_DE;ARTIGO;SECRETARIA;LISTA;DOM\n'

for item in list_items:
    s_item = item.strip().split(' ')
    if 'RF NOME DIAS A_PARTIR_DE ARTIGO' in item:
        SECRETARIA = list_items[i-1]
    elif len(s_item) >= 5 and s_item[0][0].isnumeric() and s_item[-2][0].isnumeric():
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-2])
        str_list_items += f'{rf};{vinc};{nome};{s_item[-2]};{s_item[-1]};{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
    elif s_item[0][-1].isnumeric() and VALIDA_LINHA == False:
        linha = s_item
        VALIDA_LINHA = True
    elif VALIDA_LINHA == True:
        linha += f' {item}'
        s_item = item.split(' ')
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-2])
        str_list_items += f'{rf};{vinc};{nome};{s_item[-2]};{s_item[-1]};{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
        VALIDA_LINHA = False
    i += 1

In [135]:
RELACAO = str('_').join(RELACAO.replace('/','_').split(' '))
save_file(SAVE_DIR,str_list_items,RELACAO,'csv')

'c:\\Projetos\\scrapy_dom\\outputs_files\\2022-05-31\\RELAÇÃO_DE_FALTA_DO_SERVIDOR_À_PERÍCIA.csv'

#### RELAÇÃO DE SERVIDORES SUBMETIDOS À AVALIAÇÃO DE CAPACIDADE LABORATIVA COM CAPACIDADE LABORATIVA PARA A SUA FUNÇÃO

In [136]:
RELACAO = 'RELAÇÃO DE SERVIDORES SUBMETIDOS À AVALIAÇÃO DE CAPACIDADE LABORATIVA COM CAPACIDADE LABORATIVA PARA A SUA FUNÇÃO'

if RELACAO in licencas:
    list_items = rl_lm[licencas[RELACAO]:listas[listas.index(licencas[RELACAO])+1]]
else:
    list_items = []
#list_items

In [137]:
'''
    COLUNAS:
        1. RF           : numeric(7) s_item[0][0]
        2. VINC         : numeric(1) s_item[0][1]
        3. NOME         : str()      s_item[1:]
'''

i = 0
SECRETARIA = ''
VALIDA_LINHA = ''
str_list_items = 'RF;VINC;NOME;SECRETARIA;LISTA;DOM\n'

for item in list_items:
    s_item = item.strip().split(' ')
    if 'RF NOME' in item:
        SECRETARIA = list_items[i-1]
    elif len(s_item) >= 5 and s_item[0][0].isnumeric():
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:])
        str_list_items += f'{rf};{vinc};{nome};{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
    elif s_item[0][-1].isnumeric() and VALIDA_LINHA == False:
        linha = s_item
        VALIDA_LINHA = True
    elif VALIDA_LINHA == True:
        linha += f' {item}'
        s_item = item.split(' ')
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:])
        str_list_items += f'{rf};{vinc};{nome};{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
        VALIDA_LINHA = False
    i += 1

In [138]:
RELACAO = str('_').join(RELACAO.replace('/','_').split(' '))
save_file(SAVE_DIR,str_list_items,RELACAO,'csv')

'c:\\Projetos\\scrapy_dom\\outputs_files\\2022-05-31\\RELAÇÃO_DE_SERVIDORES_SUBMETIDOS_À_AVALIAÇÃO_DE_CAPACIDADE_LABORATIVA_COM_CAPACIDADE_LABORATIVA_PARA_A_SUA_FUNÇÃO.csv'

#### ENGLOBAMENTO DE LICENÇAS MÉDICAS FACE PORTARIA 11/84

In [139]:
RELACAO = 'ENGLOBAMENTO DE LICENÇAS MÉDICAS FACE PORTARIA 11/84'

if RELACAO in licencas:
    list_items = rl_lm[licencas[RELACAO]:listas[listas.index(licencas[RELACAO])+1]]
else:
    list_items = []
list_items

['ENGLOBAMENTO DE LICENÇAS MÉDICAS FACE PORTARIA 11/84',
 'SECRETARIA MUNICIPAL DA SAUDE',
 'RF NOME DIAS A_PARTIR_DE ARTIGO',
 '7263384/1 WILSON ROBERTO DE SOUZA BARBOSA 027 07/01/2022 143',
 '7837836/1 RAIMUNDA CASSIA BARRETO SILVA 084 05/01/2022 143',
 '7895739/1 ADRIANE ALVES DOS SANTOS 025 07/04/2022 143',
 '8190909/1 MARIA CLAUDIA GALBIATTI ABREU CALIL 003 30/01/2022 143',
 '8190909/3 MARIA CLAUDIA GALBIATTI ABREU CALIL 003 30/01/2022 143',
 '8309469/2 ELISABETE THOMAZ DAMIAO CARDOSO 082 05/02/2022 143',
 '8319758/2 KAMILLA PERRI LOPES ESTEVAM 059 01/02/2022 143',
 '8329303/2 MARIA FLAVIA OLIVEIRA PEQUENO TODUCZ 007 15/12/2021 143',
 'SECRETARIA MUNICIPAL DE EDUCACAO',
 'RF NOME DIAS A_PARTIR_DE ARTIGO',
 '5998476/4 SIDNEA FRANCISCA DA SILVA 058 05/02/2022 143',
 '6866786/2 VIGANOR ROCHA BATISTA 088 05/02/2022 143',
 '6910254/1 COSME CARDOSO SANCHES 088 24/01/2022 143',
 '6910254/2 COSME CARDOSO SANCHES 088 24/01/2022 143',
 '6936164/2 ALEKSANDER SANTANA DE VASCONCELLO 014 22/09/

In [140]:
'''
    COLUNAS:
        1. RF           : numeric(7) s_item[0][0]
        2. VINC         : numeric(1) s_item[0][1]
        3. NOME         : str()      s_item[1:-3]
        4. DIAS         : numeric()  s_item[-3]
        5. A_PARTIR_DE  : date()     s_item[-2]
        6. ARTIGO       : str()      s_item[-1]
'''

i = 0
SECRETARIA = ''
VALIDA_LINHA = ''
str_list_items = 'RF;VINC;NOME;DIAS;A_PARTIR_DE;ARTIGO;SECRETARIA;LISTA;DOM\n'

for item in list_items:
    s_item = item.strip().split(' ')
    if 'RF NOME DIAS A_PARTIR_DE ARTIGO' in item:
        SECRETARIA = list_items[i-1]
    elif len(s_item) >= 5 and s_item[0][0].isnumeric() and s_item[-3][0].isnumeric() and s_item[-2][0].isnumeric():
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-3])
        str_list_items += f'{rf};{vinc};{nome};{s_item[-3]};{s_item[-2]};{s_item[-1]};{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
    elif s_item[0][-1].isnumeric() and VALIDA_LINHA == False:
        linha = s_item
        VALIDA_LINHA = True
    elif VALIDA_LINHA == True:
        linha += f' {item}'
        s_item = item.split(' ')
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-1])
        str_list_items += f'{rf};{vinc};{nome};;{s_item[-1]};;{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
        VALIDA_LINHA = False
    i += 1

In [141]:
RELACAO = str('_').join(RELACAO.replace('/','_').split(' '))
COMPLEMENTO = str('_').join('RELAÇÃO DE LICENÇAS MÉDICAS RETIFICADAS EM FUNÇÃO DE:'.replace(':','_').split(' '))
save_file(SAVE_DIR,str_list_items,f'{COMPLEMENTO}{RELACAO}','csv')

'c:\\Projetos\\scrapy_dom\\outputs_files\\2022-05-31\\RELAÇÃO_DE_LICENÇAS_MÉDICAS_RETIFICADAS_EM_FUNÇÃO_DE_ENGLOBAMENTO_DE_LICENÇAS_MÉDICAS_FACE_PORTARIA_11_84.csv'

#### RELAÇÃO DE ALTAS DO ACIDENTE DO TRABALHO E DA DOENÇA DO TRABALHO

In [283]:
RELACAO = 'RELAÇÃO DE ALTAS DO ACIDENTE DO TRABALHO E DA DOENÇA DO TRABALHO'

if RELACAO in licencas:
    list_items = rl_lm[licencas[RELACAO]:listas[listas.index(licencas[RELACAO])+1]]
else:
    list_items = []
#list_items

In [284]:
'''
    COLUNAS:
        1. RF           : numeric(7) s_item[0][0]
        2. VINC         : numeric(1) s_item[0][1]
        3. NOME         : str()      s_item[1:-1]
        4. DIAS         : null
        5. A_PARTIR_DE  : date()     s_item[-1]
        6. ARTIGO       : null
'''

i = 0
SECRETARIA = ''
VALIDA_LINHA = ''
str_list_items = 'RF;VINC;NOME;DIAS;A_PARTIR_DE;ARTIGO;SECRETARIA;LISTA;DOM\n'

for item in list_items:
    s_item = item.strip().split(' ')
    if 'RF NOME A_PARTIR_DE' in item:
        SECRETARIA = list_items[i-1]
    elif len(s_item) >= 4 and s_item[0][0].isnumeric() and s_item[-1][0].isnumeric():
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-1])
        str_list_items += f'{rf};{vinc};{nome};;{s_item[-1]};;{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
    elif s_item[0][-1].isnumeric() and VALIDA_LINHA == False:
        linha = s_item
        VALIDA_LINHA = True
    elif VALIDA_LINHA == True:
        linha += f' {item}'
        s_item = item.split(' ')
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-1])
        str_list_items += f'{rf};{vinc};{nome};;{s_item[-1]};;{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
        VALIDA_LINHA = False
    i += 1

In [285]:
RELACAO = str('_').join(RELACAO.replace('/','_').split(' '))
save_file(SAVE_DIR,str_list_items,f'{COMPLEMENTO}{RELACAO}','csv')

'c:\\Projetos\\scrapy_dom\\outputs_files\\2022-05-31\\RELAÇÃO_DE_LICENÇAS_MÉDICAS_RETIFICADAS_EM_FUNÇÃO_DE_RELAÇÃO_DE_ALTAS_DO_ACIDENTE_DO_TRABALHO_E_DA_DOENÇA_DO_TRABALHO.csv'

#### RELAÇÃO DE ALTAS DO ACIDENTE DO TRABALHO E DA DOENÇA DO TRABALHO :

In [297]:
RELACAO = 'RELAÇÃO DE ALTAS DO ACIDENTE DO TRABALHO E DA DOENÇA DO TRABALHO :'

if RELACAO in licencas:
    list_items = rl_lm[licencas[RELACAO]:listas[listas.index(licencas[RELACAO])+1]]
else:
    list_items = []
list_items

['RELAÇÃO DE ALTAS DO ACIDENTE DO TRABALHO E DA DOENÇA DO TRABALHO :',
 'APÓS TRANSFORMAÇÃO DO ARTIGO DA LICENÇA MÉDICA',
 'SECRETARIA MUNICIPAL DE EDUCACAO',
 'RF NOME A_PARTIR_DE',
 '6822576/1 ANA MARIA CARNAVAROLO NOVAES DA SILVA 08/04/2022']

In [298]:
'''
    COLUNAS:
        1. RF           : numeric(7) s_item[0][0]
        2. VINC         : numeric(1) s_item[0][1]
        3. NOME         : str()      s_item[1:-1]
        4. DIAS         : null
        5. A_PARTIR_DE  : date()     s_item[-1]
        6. ARTIGO       : null
'''
LISTAS = [
            'APÓS TRANSFORMAÇÃO DO ARTIGO DA LICENÇA MÉDICA',
        ]

i = 0
SECRETARIA = ''
VALIDA_LINHA = ''
str_list_items = 'RF;VINC;NOME;DIAS;A_PARTIR_DE;ARTIGO;SECRETARIA;LISTA;DOM\n'

for item in list_items:
    s_item = item.strip().split(' ')
    if item in LISTAS:
            RELACAO2 = f'{RELACAO} {item}'
    if 'RF NOME A_PARTIR_DE' in item:
        SECRETARIA = list_items[i-1]
    elif len(s_item) >= 4 and s_item[0][0].isnumeric() and s_item[-1][0].isnumeric():
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-1])
        str_list_items += f'{rf};{vinc};{nome};;{s_item[-1]};;{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
    elif s_item[0][-1].isnumeric() and VALIDA_LINHA == False:
        linha = s_item
        VALIDA_LINHA = True
    elif VALIDA_LINHA == True:
        linha += f' {item}'
        s_item = item.split(' ')
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-1])
        str_list_items += f'{rf};{vinc};{nome};;{s_item[-1]};;{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
        VALIDA_LINHA = False
    i += 1

In [299]:
RELACAO = str('_').join(RELACAO.replace('/','_').replace(':','_').split(' '))
save_file(SAVE_DIR,str_list_items,f'{COMPLEMENTO}{RELACAO}','csv')

'c:\\Projetos\\scrapy_dom\\outputs_files\\2022-05-31\\RELAÇÃO_DE_LICENÇAS_MÉDICAS_RETIFICADAS_EM_FUNÇÃO_DE_RELAÇÃO_DE_ALTAS_DO_ACIDENTE_DO_TRABALHO_E_DA_DOENÇA_DO_TRABALHO__.csv'

#### RELAÇÃO DE CONCESSÃO DE NEXO DE ACIDENTE DO TRABALHO DEFERIDO

In [268]:
RELACAO = 'RELAÇÃO DE CONCESSÃO DE NEXO DE ACIDENTE DO TRABALHO DEFERIDO'

if RELACAO in licencas:
    list_items = rl_lm[licencas[RELACAO]:listas[listas.index(licencas[RELACAO])+1]]
else:
    list_items = []
list_items

['RELAÇÃO DE CONCESSÃO DE NEXO DE ACIDENTE DO TRABALHO DEFERIDO',
 'SECRETARIA MUNICIPAL DE EDUCACAO',
 'RF NOME A_PARTIR_DE',
 '8408904/1 ALINE CAPONI GUSSI 12/04/2022']

In [269]:
'''
    COLUNAS:
        1. RF           : numeric(7) s_item[0][0]
        2. VINC         : numeric(1) s_item[0][1]
        3. NOME         : str()      s_item[1:-1]
        4. DIAS         : null
        5. A_PARTIR_DE  : date()     s_item[-1]
        6. ARTIGO       : null
'''

i = 0
SECRETARIA = ''
VALIDA_LINHA = ''
str_list_items = 'RF;VINC;NOME;DIAS;A_PARTIR_DE;ARTIGO;SECRETARIA;LISTA;DOM\n'

for item in list_items:
    s_item = item.strip().split(' ')
    if 'RF NOME DIAS A_PARTIR_DE ARTIGO' in item:
        SECRETARIA = list_items[i-1]
    elif len(s_item) >= 5 and s_item[0][0].isnumeric() and s_item[-1][0].isnumeric():
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-1])
        str_list_items += f'{rf};{vinc};{nome};;{s_item[-1]};;{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
    elif s_item[0][-1].isnumeric() and VALIDA_LINHA == False:
        linha = s_item
        VALIDA_LINHA = True
    elif VALIDA_LINHA == True:
        linha += f' {item}'
        s_item = item.split(' ')
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-1])
        str_list_items += f'{rf};{vinc};{nome};;{s_item[-1]};;{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
        VALIDA_LINHA = False
    i += 1

In [270]:
RELACAO = str('_').join(RELACAO.replace('/','_').split(' '))
save_file(SAVE_DIR,str_list_items,f'{COMPLEMENTO}{RELACAO}','csv')

'c:\\Projetos\\scrapy_dom\\outputs_files\\2022-05-31\\RELAÇÃO_DE_LICENÇAS_MÉDICAS_RETIFICADAS_EM_FUNÇÃO_DE_RELAÇÃO_DE_CONCESSÃO_DE_NEXO_DE_ACIDENTE_DO_TRABALHO_DEFERIDO.csv'

#### RELAÇÃO DE CONCESSÃO DE NEXO DE ACIDENTE DO TRABALHO DEFERIDO EM GRAU DE RECURSO

In [271]:
RELACAO = 'RELAÇÃO DE CONCESSÃO DE NEXO DE ACIDENTE DO TRABALHO DEFERIDO EM GRAU DE RECURSO'

if RELACAO in licencas:
    list_items = rl_lm[licencas[RELACAO]:listas[listas.index(licencas[RELACAO])+1]]
else:
    list_items = []
list_items

['RELAÇÃO DE CONCESSÃO DE NEXO DE ACIDENTE DO TRABALHO DEFERIDO EM GRAU DE RECURSO',
 'SECRETARIA MUNICIPAL DA SAUDE',
 'RF NOME A_PARTIR_DE',
 '7172401/1 JANIO AZEREDO 18/02/2022',
 'SECRETARIA MUNICIPAL DE EDUCACAO',
 'RF NOME A_PARTIR_DE',
 '7813643/2 REGIANE CRISTINA BISPO TEODORO 12/04/2022',
 '7813643/3 REGIANE CRISTINA BISPO TEODORO 12/04/2022',
 'SECRETARIA MUNICIPAL DE SEGURANCA URBANA',
 'RF NOME A_PARTIR_DE',
 '7717695/1 ROGERIO JOSE ALEXANDRE 12/03/2022']

In [272]:
'''
    COLUNAS:
        1. RF           : numeric(7) s_item[0][0]
        2. VINC         : numeric(1) s_item[0][1]
        3. NOME         : str()      s_item[1:-1]
        4. DIAS         : null
        5. A_PARTIR_DE  : date()     s_item[-1]
        6. ARTIGO       : null
'''

i = 0
SECRETARIA = ''
VALIDA_LINHA = ''
str_list_items = 'RF;VINC;NOME;DIAS;A_PARTIR_DE;ARTIGO;SECRETARIA;LISTA;DOM\n'

for item in list_items:
    s_item = item.strip().split(' ')
    if 'RF NOME A_PARTIR_DE' in item:
        SECRETARIA = list_items[i-1]
    elif len(s_item) >= 4 and s_item[0][0].isnumeric() and s_item[-1][0].isnumeric():
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-1])
        str_list_items += f'{rf};{vinc};{nome};;{s_item[-1]};;{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
    elif s_item[0][-1].isnumeric() and VALIDA_LINHA == False:
        linha = s_item
        VALIDA_LINHA = True
    elif VALIDA_LINHA == True:
        linha += f' {item}'
        s_item = item.split(' ')
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-1])
        str_list_items += f'{rf};{vinc};{nome};;{s_item[-1]};;{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
        VALIDA_LINHA = False
    i += 1

In [273]:
RELACAO = str('_').join(RELACAO.replace('/','_').split(' '))
save_file(SAVE_DIR,str_list_items,f'{COMPLEMENTO}{RELACAO}','csv')

'c:\\Projetos\\scrapy_dom\\outputs_files\\2022-05-31\\RELAÇÃO_DE_LICENÇAS_MÉDICAS_RETIFICADAS_EM_FUNÇÃO_DE_RELAÇÃO_DE_CONCESSÃO_DE_NEXO_DE_ACIDENTE_DO_TRABALHO_DEFERIDO_EM_GRAU_DE_RECURSO.csv'

#### RELAÇÃO DE CONCESSÃO DE NEXO DE ACIDENTE DO TRABALHO INDEFERIDO

In [275]:
RELACAO = 'RELAÇÃO DE CONCESSÃO DE NEXO DE ACIDENTE DO TRABALHO INDEFERIDO'

if RELACAO in licencas:
    list_items = rl_lm[licencas[RELACAO]:listas[listas.index(licencas[RELACAO])+1]]
    #print(licencas[RELACAO])
    #print(listas[listas.index(licencas[RELACAO])+1])
else:
    list_items = []
#list_items

In [276]:
'''
    COLUNAS:
        1. RF           : numeric(7) s_item[0][0]
        2. VINC         : numeric(1) s_item[0][1]
        3. NOME         : str()      s_item[1:-1]
        4. DIAS         : null
        5. A_PARTIR_DE  : date()     s_item[-1]
        6. ARTIGO       : null
'''

i = 0
SECRETARIA = ''
VALIDA_LINHA = ''
str_list_items = 'RF;VINC;NOME;DIAS;A_PARTIR_DE;ARTIGO;SECRETARIA;LISTA;DOM\n'

for item in list_items:
    s_item = item.strip().split(' ')
    if 'RF NOME A_PARTIR_DE' in item:
        SECRETARIA = list_items[i-1]
    elif len(s_item) >= 4 and s_item[0][0].isnumeric() and s_item[-1][0].isnumeric():
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-1])
        str_list_items += f'{rf};{vinc};{nome};;{s_item[-1]};;{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
    elif s_item[0][-1].isnumeric() and VALIDA_LINHA == False:
        linha = s_item
        VALIDA_LINHA = True
    elif VALIDA_LINHA == True:
        linha += f' {item}'
        s_item = item.split(' ')
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-1])
        str_list_items += f'{rf};{vinc};{nome};;{s_item[-1]};;{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
        VALIDA_LINHA = False
    i += 1

In [277]:
RELACAO = str('_').join(RELACAO.replace('/','_').split(' '))
save_file(SAVE_DIR,str_list_items,f'{COMPLEMENTO}{RELACAO}','csv')

'c:\\Projetos\\scrapy_dom\\outputs_files\\2022-05-31\\RELAÇÃO_DE_LICENÇAS_MÉDICAS_RETIFICADAS_EM_FUNÇÃO_DE_RELAÇÃO_DE_CONCESSÃO_DE_NEXO_DE_ACIDENTE_DO_TRABALHO_INDEFERIDO.csv'